<a href="https://colab.research.google.com/github/puc26/myProject/blob/main/AttendanceCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 出勤規則：
1. 標準上班時間：08:30 - 18:00
2. 彈性上班時間：09:00 - 18:30
3. 午休時間：12:00 - 13:30
4. 超過上班時間半小時後開始計算加班，以每半小時為單位計算
5. 程式返回值：總出勤時間、遲到時間、早退時間、請假時間、加班時間

### 使用說明：
資料格式：直接使用數字，開頭為零請捨去。（ex. 08:30 -> 830, 17:30 -> 1730）  
startTime：上班打卡時間  
endTime：下班打卡時間  
leaveTimes：請假時間，可多個時段。（ex. leaveTimes = ((830, 1000), (1600, 1800))）

# 定義演算式

In [4]:
import datetime
import pandas as pd

In [11]:
# 合併上下班時段及請假時段，排除午休時段，返回上午及下午總時段
def merge_times(startTime, endTime, leaveTimes):
    # 定義時間格式
    formatStr = '%H%M'

    mergedTimes = []

    # 將上班時間範圍加入合併後的列表
    mergedTimes.append((startTime, endTime))

    # 將請假時間範圍加入合併後的列表
    for leaveStart, leaveEnd in leaveTimes:
        mergedTimes.append((leaveStart, leaveEnd))

    # 根據時間的起始時間進行排序
    mergedTimes.sort(key=lambda x: x[0])

    # 午休時段
    noonStart = '1200'
    noonEnd = '1330'

    # 合併重疊的時間範圍
    mergedResult = []
    currentStart = None
    currentEnd = None

    for startTime, endTime in mergedTimes:
        if currentStart is None:
            currentStart = startTime
            currentEnd = endTime
        elif startTime <= currentEnd:
            currentEnd = max(currentEnd, endTime)
        else:
            mergedResult.append((currentStart, currentEnd))
            currentStart = startTime
            currentEnd = endTime

    if currentStart is not None:
        mergedResult.append((currentStart, currentEnd))

    # 分割上午和下午的合併時段
    morningResult = []
    afternoonResult = []
    vacancy = []

    for startTime, endTime in mergedResult:
        if endTime <= noonStart:
            morningResult.append((startTime, endTime))
        elif startTime >= noonEnd:
            afternoonResult.append((startTime, endTime))
        else:
            if startTime < noonStart:
                morningResult.append((startTime, noonStart))
            if endTime > noonEnd:
                afternoonResult.append((noonEnd, endTime))

    # 加入空缺時段
    for i in range(len(morningResult) - 1):
      endTime = morningResult[i][-1]
      startTime = morningResult[i+1][0]
      if endTime != startTime:
        vacancy.append((endTime, startTime))

    for i in range(len(afternoonResult) - 1):
      endTime = afternoonResult[i][-1]
      startTime = afternoonResult[i+1][0]
      if endTime != startTime:
        vacancy.append((endTime, startTime))

    # 加入鄰近午休時間的空缺時段
    if len(morningResult) > 0 and morningResult[-1][1] != '1200':
      vacancy.append((morningResult[-1][1], '1200'))
    if len(afternoonResult) > 0 and afternoonResult[0][0] != '1330':
      vacancy.append(('1330', afternoonResult[0][0]))

    # 加入未上班時段
    if len(morningResult) == 0:
      vacancy.append(('0830', '1200'))
    if len(afternoonResult) == 0:
      vacancy.append(('1330', '1800'))

    return morningResult, afternoonResult, vacancy


In [12]:
# 定義出勤計算機
def work_times(startTime, endTime, leaveTimes):
  # 轉換使用者輸入資料格式
  startTime = str(startTime).zfill(4)
  endTime = str(endTime).zfill(4)
  leaveTimes = [(str(startTime).zfill(4), str(endTime).zfill(4)) for startTime, endTime in leaveTimes]

  # 定義時間格式
  formatStr = '%H%M'

  # 初始化參數
  late = 0
  earlyLeave = 0
  overtime = 0
  totalLeave = 0

  # 正常上班時間範圍
  morningWorkStart = datetime.datetime.strptime('0830', formatStr)
  morningWorkEnd = datetime.datetime.strptime('1200', formatStr)
  afternoonWorkStart = datetime.datetime.strptime('1330', formatStr)
  afternoonWorkEnd = datetime.datetime.strptime('1800', formatStr)
  # 彈性上下班時間的範圍
  flexMorningWorkStart = datetime.datetime.strptime('0900', formatStr)
  flexAfternoonWorkEnd = datetime.datetime.strptime('1830', formatStr)

  # 檢查是否有空缺時段
  morningResult, afternoonResult, vacancy = merge_times(startTime, endTime, leaveTimes)
  if len(vacancy) > 0:
    return '上班時段有空缺時段，請檢查時段： ', vacancy, ' !!'

  # 上下午時段皆存在
  if len(morningResult) != 0 and len(afternoonResult) != 0:
    start = datetime.datetime.strptime(morningResult[0][0], formatStr)
    end = datetime.datetime.strptime(afternoonResult[-1][-1], formatStr)

    # 計算遲到與早退
    if start <= morningWorkStart: # 早於上班時間，起始時間等於正常上班時間
      start = morningWorkStart
    elif morningWorkStart < start <= flexMorningWorkStart: # 開始上班於彈性上班範圍內
      afternoonWorkEnd += (start - morningWorkStart) # 重新正常下班時間
    else: # 開始上班時間超過彈性上班時間
      afternoonWorkEnd = flexAfternoonWorkEnd

    late = (start - flexMorningWorkStart).total_seconds() // 60 if start > flexMorningWorkStart else 0
    early_leave = (afternoonWorkEnd - end).total_seconds() // 60 if end < afternoonWorkEnd else 0

  # 上午時段存在、下午時段不存在
  elif len(morningResult) != 0 and len(afternoonResult) == 0:
    start = datetime.datetime.strptime(morningResult[0][0], format_str)
    end = datetime.datetime.strptime(morningResult[-1][-1], format_str)

    # 計算遲到與早退
    if start <= morningWorkStart: # 開始上班小於等於正常上班時間
      start = morningWorkStart
    elif morningWorkStart < start <= flexMorningWorkStart: # 開始上班於彈性上班範圍內
      afternoonWorkEnd += (start - morningWorkStart) # 重新定義正常下班時間
    else: # 開始上班時間超過彈性上班時間
      afternoonWorkEnd = flexAfternoonWorkEnd # 重新定義彈性下班時間

    late = (start - flexMorningWorkStart).total_seconds() // 60 if start > flexMorningWorkStart else 0
    earlyLeave = (afternoonWorkEnd - end).total_seconds() // 60 if end < afternoonWorkEnd else 0

  # 上午時段不存在、下午時段存在
  elif len(morningResult) == 0 and len(afternoonResult) != 0:
    start = datetime.datetime.strptime(afternoonResult[0][0], formatStr)
    end = datetime.datetime.strptime(afternoonResult[-1][-1], formatStr)

    # 計算遲到與早退
    if start <= afternoonWorkEnd: # 開始上班小於午休時間
      start = afternoonWorkEnd
    else: # 開始上班大於午休時間
      afternoonWorkEnd = flexAfternoonWorkEnd # 重新定義正常下班時間

    late = (start - flexMorningWorkStart).total_seconds() // 60 - 90
    earlyLeave = (afternoonWorkEnd - end).total_seconds() // 60

  # 上下午時段皆不存在
  else:
      return '不存在上班時段，請確認 !!'

  # 加班小節統計
  overtimeCount = (end - afternoonWorkEnd).total_seconds() // (datetime.timedelta(minutes=30)).total_seconds() - 1
  # 檢查融合時間下班是否晚於加班時間
  if end >= afternoonWorkEnd:
    if overtimeCount > 0:
      overtime = overtimeCount * 30 # 晚於加班時間，達到加班標準，計算加班時間
      end = afternoonWorkEnd
    else:
      end = afternoonWorkEnd # 晚於下班時間，未達加班標準
  else:
    earlyLeave = (afternoonWorkEnd - end).total_seconds() // 60 # 早於下班時間，計算早退時間

  # 轉換leave_times為datetime.datetime
  leaveTimes = [(datetime.datetime.strptime(startTime, formatStr), datetime.datetime.strptime(endTime, formatStr)) for startTime, endTime in leaveTimes]
  # 計算請假時間
  for leaveStart, leaveEnd in leaveTimes:
    leaveDuration = leaveEnd - leaveStart
    if leaveStart < morningWorkEnd and leaveEnd > afternoonWorkStart:
      leaveDuration -= datetime.timedelta(minutes=90)
    totalLeave += leaveDuration.total_seconds() // 60

  # 計算上班分鐘數
  totalMin = (end - start).total_seconds() // 60
  totalMin -= totalLeave
  if start < morningWorkEnd and end > afternoonWorkStart:
    totalMin -= 90

  # 轉換為DataFrame格式
  result = {'分鐘數': [totalMin, late, earlyLeave, totalLeave, overtime],
            '時數': [totalMin/60, late/60, earlyLeave/60, totalLeave/60, overtime/60]}
  df = pd.DataFrame(result).round(2)
  df.index = ['總時間', '遲到', '早退', '請假', '加班']

  return df

# 計算出勤時間

In [13]:
# 參數設定
startTime = 830
endTime = 1700
leaveTimes = [(1700, 1800)]

# 代入演算式（不需更改）
work_times(startTime, endTime, leaveTimes)

,分鐘數,時數
總時間,420.0,7.0
遲到,0.0,0.0
早退,0.0,0.0
請假,60.0,1.0
加班,0.0,0.0


# Author
This notebook is created by Steven Chen.  
*Jul 9 2023*  
v1.0